Let's implement context steering from scratch!

The formula for the influence function is as follows:
$$F_{C,P}(x_i) = LLM(x_i
|C,P) − LLM(x_i
|∅,P) $$

where we seek to sample some next token $x_i$ based on comparing two distributions: one based on just the prompt $P$ with no additional context and one based on a prompt with additional context $C$.

In [1]:
import random 

# Let's first simulate the forward pass of our LLM. For a prompt with context and a prompt without context, 
# let's say we received the following logprobs:
logits = [round(random.uniform(-5, 0), 2) for _ in range(10)]
logits_no_context = [round(random.uniform(-5, 0), 2) for _ in range(10)]

print(f'Logits for LLM(l1): {logits_no_context}')
print(f'Logits for LLM(l1 + c): {logits}')

Logits for LLM(l1): [-3.37, -4.14, -1.43, -4.45, -4.86, -4.54, -1.95, -4.31, -2.82, -0.61]
Logits for LLM(l1 + c): [-2.97, -3.73, -3.27, -0.92, -4.58, -0.57, -3.31, -2.24, -1.25, -2.51]


In [2]:
import torch
import torch.nn.functional as F

# For numerically stability reasons, we take the log softmax of the logits
logprobs = F.log_softmax(torch.tensor(logits), dim=-1)
logprobs_no_context = F.log_softmax(torch.tensor(logits_no_context), dim=-1)

# We then calculate the influence as the difference between these logprobs
influence = logprobs - logprobs_no_context

influence

tensor([ 0.0065,  0.0165, -2.2335,  3.1365, -0.1135,  3.5765, -1.7535,  1.6765,
         1.1765, -2.2935])

Next, let's calculate the subsequent probability distribution to sample from using context steering. To get the next token probabilities for some token $x$, we use the following calculation. 

$$ CoS_{\lambda}(x_i
|C,P) = LLM(x_i
|C,P) + \lambda · F_{C,P}(x_i)
 $$

Recall that we determine the level of contextual influence by tweaking lambda.

In [3]:
def apply_lambda(lmbda, logprobs, influence):
    # Apply the formula from above
    cos_distribution = logprobs + lmbda * influence

    # Normalize probabilities
    cos_distribution = F.log_softmax(cos_distribution, dim=-1)

    # If we want to work with probabilities instead of log probabilities, exponentiate
    cos_distribution = torch.exp(cos_distribution)

    return cos_distribution

Try out a few different values of influence:

In [4]:
for lmbda in [-1.0, 1.0, 3.0]:
    dist = apply_lambda(lmbda, logprobs, influence)
    print(f'Lambda: {lmbda}, Distribution: {dist.tolist()}')

Lambda: -1.0, Distribution: [0.03188985213637352, 0.014765418134629726, 0.22191348671913147, 0.010829651728272438, 0.007187098730355501, 0.00989755429327488, 0.13193213939666748, 0.012457062490284443, 0.05527316406369209, 0.503854513168335]
Lambda: 1.0, Distribution: [0.0016660697292536497, 0.0007869956316426396, 0.00013139699876774102, 0.29603198170661926, 0.0002953687508124858, 0.6522760391235352, 0.00020402116933837533, 0.01836538314819336, 0.02997811883687973, 0.00026460099616087973]
Lambda: 3.0, Distribution: [1.7026394516506116e-06, 8.205175845432677e-07, 1.5218647453352219e-09, 0.15828946232795715, 2.3744557609006733e-07, 0.8408601880073547, 6.171463340365335e-09, 0.0005296311574056745, 0.00031804124591872096, 2.7181086270644528e-09]


In practice, you can use the batched version of this code via the `apply_cos` function in `cos/core.py`.